In [3]:
import pandas as pd
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
from scipy.optimize import minimize
import scipy
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.1/Resources/'
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import datetime
from rpy2.robjects.vectors import FloatVector
import multiprocessing

np.random.seed(441)

# Define the model that generates pair simulations.
yuima = importr("yuima")
n_ou_sim_string = """
n_sim_ou = function(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length){

  set.seed(random_seed)

  drift = c("mu11-mu12*X1", "mu21-mu22*X2")
  diffusion = matrix(c("exp(sigma11)", "exp(sigma12)", "exp(sigma21)", "exp(sigma22)"), 2, 2, byrow=TRUE)
  ou_model = setModel(drift=drift, diffusion=diffusion, 
                        time.variable = "t",
                        state.var=c("X1","X2"), solve.variable=c("X1","X2"))

  newsamp = setSampling(Initial=T0, Terminal=T, n=length)

  n_sim_ou_data = data.frame(matrix(nrow=length+1, ncol=2*num_sim))
  for (i in 1:num_sim){
    ou_sim = simulate(ou_model, 
                      true.par=list(
                        mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22, 
                        sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22), 
                      xinit=xinit_vec[i], sampling=newsamp)
    original_data = ou_sim@data@original.data
    one_sim_ou = data.frame(original_data[,1], original_data[,2])
    colnames(one_sim_ou) = c('series1', 'series2')
    n_sim_ou_data[, (2*i-1):(2*i)] = one_sim_ou
  }
  return(n_sim_ou_data)
}
"""

n_ou_sim = SignatureTranslatedAnonymousPackage(n_ou_sim_string, "n_ou_sim")
def n_ou_simulation(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length):
    """num_sim simulations of bivariate Ornstein-Uhlenbeck process,
    length = length of one series
    """
    n_ou_sim_data = pd.DataFrame(
        n_ou_sim.n_sim_ou(random_seed=random_seed, num_sim=num_sim,
                              mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22,
                              sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22,
                              xinit_vec=xinit_vec, T0=T0, T=T, length=length)).transpose()
    return n_ou_sim_data

def price_to_log_price(n_price):
    return(np.log(n_price))

def log_price_to_price(n_log_price):
    return(np.exp(n_log_price))

def price_to_return(n_price):
    n_return = pd.DataFrame()
    for i in range(n_price.shape[1]):
        ith_column_price_series = n_price.iloc[:, i]
        n_return = pd.concat([n_return, 100 * (np.log(ith_column_price_series[1:].values) - np.log(ith_column_price_series[:-1]))], axis=1)
    return n_return

def log_price_to_return(n_log_price):
    n_real_return = pd.DataFrame()
    for i in range(n_log_price.shape[1]):
        ith_column_price_series = n_log_price.iloc[:, i]
        n_real_return = pd.concat([n_real_return, 100 * (ith_column_price_series[1:].values - ith_column_price_series[:-1])], axis=1)
    return n_real_return

def cal_stats(n_return, n_price=None):
    # (different expressions of calculation from intro to stat finance)
    # 4 statistics
    return_series1 = n_return.iloc[:, ::2]
    return_series2 = n_return.iloc[:, 1::2]
    mean1 = return_series1.mean(axis=0).values
    sd1 = return_series1.std(axis=0).values
    mean2 = return_series2.mean(axis=0).values
    sd2 = return_series2.std(axis=0).values
    stats_data = pd.DataFrame([mean1, mean2, sd1, sd2])
    stats_data = stats_data.transpose()
    stats_data.columns = [
        'return_mean1', 'return_mean2',
        'return_sd1', 'return_sd2']
    return stats_data

def loss_function(params):
    """n_real_stats is a global amount calculated outside the function"""
    params = FloatVector(params)
    print(123)
    print(params)
    moment_loss = pd.DataFrame().reindex_like(real_stats)


    n_sim_log_price = n_ou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=mu11, mu12=params[0], mu21=mu21, mu22=params[1],
        sigma11=params[2], sigma12=sigma12, sigma21=sigma21, sigma22=params[3],
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    n_sim_price = log_price_to_price(n_sim_log_price)
    n_sim_return = price_to_return(n_sim_price)
    n_sim_stats = cal_stats(n_sim_return)


    for i in range(n_real_stats.shape[0]):
        for j in range(n_real_stats.shape[1]):
            moment_loss.iloc[i, j] = np.sqrt((n_real_stats.iloc[i, j] - n_sim_stats.iloc[i, j])**2)
    sum_all = np.sum(moment_loss)
    print(sum_all)
    print(np.sum(sum_all))
    print('---')

    return np.sum(sum_all)



real_price = pd.read_csv("sp500_20180101_20181231_pair_prices.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv("sp500_20180101_20181231_pair_returns.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=None)

mu11, mu21, sigma12, sigma21 = 0, 0, -1000, -1000

xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
num_sim, T0, T, length = real_stats.shape[0], 0, 1, real_price.shape[0]

n_real_stats = real_stats



initial0 = [1, 1, -1, -1]



def con1(x):
    
    print(456)
    out = x[0] - 2
    return out
    
    
begin_time = datetime.datetime.now()
res = minimize(loss_function, initial0, method='SLSQP',
               bounds=[(0, None), (0, None), (None, None), (None, None)],
               constraints={'type': 'ineq', 'fun': con1})
print(res.x)

time = datetime.datetime.now() - begin_time
print(time)

params = (res.x)
loss = loss_function((params))
print(loss)

456
123
[1]  1  1 -1 -1

return_mean1    266.810737
return_mean2    254.309517
return_sd1      173.505310
return_sd2      175.074364
dtype: float64
869.6999289105129
---
123
[1]  1  1 -1 -1

return_mean1    268.511516
return_mean2    255.209460
return_sd1      174.723935
return_sd2      175.052979
dtype: float64
873.4978900938063
---
123
[1]  1  1 -1 -1

return_mean1    267.706252
return_mean2    254.597386
return_sd1      177.734501
return_sd2      174.782313
dtype: float64
874.8204519233661
---
123
[1]  1  1 -1 -1

return_mean1    265.860130
return_mean2    257.082894
return_sd1      173.068393
return_sd2      170.487935
dtype: float64
866.4993529622955
---
123
[1]  1  1 -1 -1

return_mean1    268.805459
return_mean2    254.862270
return_sd1      172.134557
return_sd2      173.028149
dtype: float64
868.8304352856035
---
456
456
456
456
456
456
123
[1]         0         0 214787015  58350728



/opt/homebrew/anaconda3/envs/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log
/opt/homebrew/anaconda3/envs/venv37/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]        0.9        0.9 21478700.6  5835071.9

return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]       0.99       0.99 2147869.16  583506.29

return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]      0.999      0.999 214786.016  58349.729

return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]     0.9999     0.9999 21477.7016  5834.0729

return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]    0.99999    0.99999 2146.87016  582.50729

return_mean1    inf
return_mean2    inf
return_sd1      0.0
return_sd2      0.0
dtype: float64
inf
---
456
123
[1]   0.999999   0.999999 213.787016  57.350729

return_mean1    inf
return_mean2    inf


In [4]:
res

     fun: 1331.1449628749738
     jac: array([ 4.04783873e+08,  3.20478298e+08,  2.18317753e+08, -3.94698794e+07])
 message: 'Inequality constraints incompatible'
    nfev: 20
     nit: 3
    njev: 2
  status: 4
 success: False
       x: array([ 1.        ,  1.        , -0.4630404 , -0.85412533])